In [1]:
import data_processing
import torch
import numpy as np
import torch_geometric
import pandas as pd

In [65]:
mask = torch.zeros(wiki.num_nodes, dtype=torch.int)
mask[:6459] = 0
mask[6459:7844] = 1
mask[7844:] = 2
indexes = torch.randperm(mask.nelement())
mask[indexes] == 0

tensor([False, False, False,  ...,  True,  True,  True])

In [3]:
u_list, i_list, ts_list, label_list = [], [], [], []
feat_l = []
idx_list = []

In [5]:
with open("data/wikipedia/raw/wikipedia.csv") as f:
    s = next(f)
    print(s)
    for idx, line in enumerate(f):
        e = line.strip().split(',')
        u = int(e[0])
        i = int(e[1])

        ts = float(e[2])
        label = int(e[3])

        feat = np.array([float(x) for x in e[4:]])

        u_list.append(u)
        i_list.append(i)
        ts_list.append(ts)
        label_list.append(label)
        idx_list.append(idx)

        feat_l.append(feat)

user_id,item_id,timestamp,state_label,comma_separated_list_of_features



In [10]:
df = pd.DataFrame({'u': u_list, 
                         'i':i_list, 
                         'ts':ts_list, 
                         'label':label_list, 
                         'idx':idx_list})
df

,u,i,ts,label,idx
0,0,0,0.0,0,0
1,1,1,36.0,0,1
2,1,1,77.0,0,2
3,2,2,131.0,0,3
4,1,1,150.0,0,4
...,...,...,...,...,...
157469,2003,632,2678155.0,0,157469
157470,3762,798,2678158.0,0,157470
157471,2399,495,2678293.0,0,157471
157472,7479,920,2678333.0,0,157472


In [32]:
feat = np.array(feat_l)
feat

array([[-0.17506251, -0.17667764, -0.93709077, ..., -0.05046383,
        -0.04144796, -0.03877482],
       [-0.17506251, -0.17667764, -0.93709077, ..., -0.05046383,
        -0.04144796, -0.03877482],
       [-0.17506251, -0.17667764, -0.93709077, ..., -0.05046383,
        -0.04144796, -0.03877482],
       ...,
       [ 0.27344377,  0.30081117,  1.12243376, ..., -0.05046383,
        -0.04144796, -0.03877482],
       [-0.17506251, -0.17667764, -0.93709077, ..., -0.05046383,
        -0.04144796, -0.03877482],
       [-0.17506251, -0.17667764, -0.93709077, ..., -0.05046383,
        -0.04144796, -0.03877482]])

In [14]:
feat.shape

(157474, 172)

In [22]:
upper_u = df.u.max() + 1
new_i = df.i + upper_u
df.i, new_i

(0           0
 1           1
 2           1
 3           2
 4           1
          ... 
 157469    632
 157470    798
 157471    495
 157472    920
 157473    495
 Name: i, Length: 157474, dtype: int64,
 0         8227
 1         8228
 2         8228
 3         8229
 4         8228
           ... 
 157469    8859
 157470    9025
 157471    8722
 157472    9147
 157473    8722
 Name: i, Length: 157474, dtype: int64)

In [24]:
new_df = df.copy()
new_df.i = new_i
new_df.u += 1
new_df.i += 1
new_df.idx += 1
new_df

,u,i,ts,label,idx
0,1,8228,0.0,0,1
1,2,8229,36.0,0,2
2,2,8229,77.0,0,3
3,3,8230,131.0,0,4
4,2,8229,150.0,0,5
...,...,...,...,...,...
157469,2004,8860,2678155.0,0,157470
157470,3763,9026,2678158.0,0,157471
157471,2400,8723,2678293.0,0,157472
157472,7480,9148,2678333.0,0,157473


In [28]:
empty = np.zeros(feat.shape[1])[np.newaxis, :]
empty.shape

(1, 172)

In [33]:
feat = np.vstack([empty, feat])
feat, feat.shape

(array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [-0.17506251, -0.17667764, -0.93709077, ..., -0.05046383,
         -0.04144796, -0.03877482],
        [-0.17506251, -0.17667764, -0.93709077, ..., -0.05046383,
         -0.04144796, -0.03877482],
        ...,
        [ 0.27344377,  0.30081117,  1.12243376, ..., -0.05046383,
         -0.04144796, -0.03877482],
        [-0.17506251, -0.17667764, -0.93709077, ..., -0.05046383,
         -0.04144796, -0.03877482],
        [-0.17506251, -0.17667764, -0.93709077, ..., -0.05046383,
         -0.04144796, -0.03877482]]),
 (157475, 172))

In [34]:
max_idx = max(new_df.u.max(), new_df.i.max())
max_idx

9227

In [35]:
rand_feat = np.zeros((max_idx + 1, feat.shape[1]))
rand_feat

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [36]:
rand_feat.shape

(9228, 172)

In [43]:
new_df.u.values.max(), new_df.i.values.max()

(8227, 9227)

In [48]:
len(set(np.unique(np.hstack([new_df.u.values, new_df.i.values]))))

9227

In [2]:
wiki = data_processing.get_jodie("Wikipedia", None)
wiki

TemporalData(src=[157474], dst=[157474], t=[157474], msg=[157474, 172], y=[157474])

In [13]:
import pandas as pd

In [14]:
y_df = pd.DataFrame(wiki.y)
y_df

,0
0,0
1,0
2,0
3,0
4,0
...,...
157469,0
157470,0
157471,0
157472,0


In [15]:
y_df.value_counts()

0    157257
1       217
dtype: int64

In [22]:
wiki.train_val_test_split()

(TemporalData(src=[110232], dst=[110232], t=[110232], msg=[110232, 172], y=[110232]),
 TemporalData(src=[23621], dst=[23621], t=[23621], msg=[23621, 172], y=[23621]),
 TemporalData(src=[23621], dst=[23621], t=[23621], msg=[23621, 172], y=[23621]))

In [23]:
wiki.t

tensor([      0,      36,      77,  ..., 2678293, 2678333, 2678373])

In [5]:
edge_time = wiki.t.view(-1,1)
edge_time

tensor([[      0],
        [     36],
        [     77],
        ...,
        [2678293],
        [2678333],
        [2678373]])

In [6]:
edge_index = torch.stack((wiki.src, wiki.dst))
edge_index

tensor([[   0,    1,    1,  ..., 2399, 7479, 2399],
        [8227, 8228, 8228,  ..., 8722, 9147, 8722]])

In [7]:
edge = torch.cat([edge_index, edge_time.view(1,-1)])
edge

tensor([[      0,       1,       1,  ...,    2399,    7479,    2399],
        [   8227,    8228,    8228,  ...,    8722,    9147,    8722],
        [      0,      36,      77,  ..., 2678293, 2678333, 2678373]])

In [8]:
import pandas as pd

In [16]:
pd.DataFrame(edge.T.numpy()).groupby(0).min()

,1,2
0,,
0,8227,0
1,8228,36
2,8229,131
3,8230,169
4,8231,218
...,...,...
8222,8555,2675481
8223,8528,2675530
8224,9205,2677219


In [20]:
pd.DataFrame(edge.T.numpy()).groupby(1).min()

,0,2
1,,
8227,0,0
8228,1,36
8229,2,131
8230,3,169
8231,4,218
...,...,...
9222,233,2396780
9223,22,2405696
9224,17,2421296


In [29]:
edge_index.T, wiki.msg

(tensor([[   0, 8227],
         [   1, 8228],
         [   1, 8228],
         ...,
         [2399, 8722],
         [7479, 9147],
         [2399, 8722]]),
 tensor([[-0.1751, -0.1767, -0.9371,  ..., -0.0505, -0.0414, -0.0388],
         [-0.1751, -0.1767, -0.9371,  ..., -0.0505, -0.0414, -0.0388],
         [-0.1751, -0.1767, -0.9371,  ..., -0.0505, -0.0414, -0.0388],
         ...,
         [ 0.2734,  0.3008,  1.1224,  ..., -0.0505, -0.0414, -0.0388],
         [-0.1751, -0.1767, -0.9371,  ..., -0.0505, -0.0414, -0.0388],
         [-0.1751, -0.1767, -0.9371,  ..., -0.0505, -0.0414, -0.0388]]))

In [30]:
edge_msg = torch.cat([edge_index.T, wiki.msg], dim=-1)
edge_msg

tensor([[ 0.0000e+00,  8.2270e+03, -1.7506e-01,  ..., -5.0464e-02,
         -4.1448e-02, -3.8775e-02],
        [ 1.0000e+00,  8.2280e+03, -1.7506e-01,  ..., -5.0464e-02,
         -4.1448e-02, -3.8775e-02],
        [ 1.0000e+00,  8.2280e+03, -1.7506e-01,  ..., -5.0464e-02,
         -4.1448e-02, -3.8775e-02],
        ...,
        [ 2.3990e+03,  8.7220e+03,  2.7344e-01,  ..., -5.0464e-02,
         -4.1448e-02, -3.8775e-02],
        [ 7.4790e+03,  9.1470e+03, -1.7506e-01,  ..., -5.0464e-02,
         -4.1448e-02, -3.8775e-02],
        [ 2.3990e+03,  8.7220e+03, -1.7506e-01,  ..., -5.0464e-02,
         -4.1448e-02, -3.8775e-02]])

In [51]:
x = torch.zeros((wiki.num_nodes - 8227, wiki.msg.shape[1]))
xx = pd.DataFrame(edge_msg.numpy()).groupby(0).mean().values[:, 1:]

In [55]:
torch.cat([torch.tensor(xx), x]).shape

torch.Size([9227, 172])

In [18]:
pd.DataFrame(edge_index.T.numpy()).groupby(0).count().index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            8217, 8218, 8219, 8220, 8221, 8222, 8223, 8224, 8225, 8226],
           dtype='int64', name=0, length=8227)

In [40]:
degree = pd.DataFrame(edge.T.numpy()).groupby(0).min().values
degree

array([[8.2270000e+03, 0.0000000e+00],
       [8.2280000e+03, 1.3440996e-05],
       [8.2290000e+03, 4.8910289e-05],
       ...,
       [9.2050000e+03, 9.9956912e-01],
       [8.9370000e+03, 9.9973530e-01],
       [8.2600000e+03, 9.9985290e-01]], dtype=float32)

In [41]:
ids = pd.DataFrame(edge_index.T.numpy()).groupby(0).count().index.values
ids

array([   0,    1,    2, ..., 8224, 8225, 8226])

In [50]:
key = {i: 0 for i in range(wiki.num_nodes)}
for i in range(len(ids)):
    key[ids[i]] = degree[i][1]
node_time = torch.tensor(list(key.values()))
node_time

tensor([0.0000e+00, 1.3441e-05, 4.8910e-05,  ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00])

In [51]:
node_time.dtype

torch.float32

In [54]:
node_out_degree = torch_geometric.utils.degree(
        edge_index[0], num_nodes=wiki.num_nodes).reshape(-1, 1)
node_out_degree

tensor([[  1.],
        [285.],
        [162.],
        ...,
        [  0.],
        [  0.],
        [  0.]])

In [55]:
torch_geometric.utils.is_undirected(edge_index)

False

In [60]:
train, val, test = wiki.train_val_test_split()
train.t, val.t, test.t

(tensor([      0,      36,      77,  ..., 1862639, 1862645, 1862652]),
 tensor([1862653, 1862659, 1862666,  ..., 2218282, 2218288, 2218288]),
 tensor([2218300, 2218303, 2218304,  ..., 2678293, 2678333, 2678373]))

In [59]:
x

NameError: name 'x' is not defined

In [61]:
val_time, test_time = list(np.quantile(wiki.t, [0.70, 0.85]))

In [62]:
val_time

1862652.1

In [63]:
test_time

2218288.5999999996

In [64]:
wiki.num_nodes

9227

In [66]:
node_time.shape

torch.Size([9227])

In [70]:
wiki.src[wiki.y == 1]

tensor([  32,  137,  515,  688,  689,  743,  690,  266,  910,  980,  637,  661,
        1120,  798, 1398, 1409, 1433, 1463, 1481, 1489, 1522, 1581,  112, 1072,
        1088, 1914, 1253, 1930, 1939, 1900, 1969, 2021, 1747, 1809, 2071, 2208,
        2212, 2150, 2250, 2284, 2299, 2222,  276, 2347, 2385, 2398, 2417, 2529,
        2635, 2656, 2737, 2760, 2789, 2735, 1234, 3019, 2719, 3074, 3089,  301,
        3177, 2931, 2235, 3234, 3248, 3279, 3018,  355, 3032, 2470, 3508,  787,
        3621, 3509, 3758, 3875, 4008, 4032, 4062, 4114, 4120, 4149, 4152, 4260,
        4379, 4485, 4475, 4502, 4500, 4497, 4582, 4257, 4594, 4663, 4703, 4898,
        4920, 1274, 4959,  354, 5022, 5106, 5111, 1798, 5172, 5233,  213,  330,
        5251, 3313, 4156, 5269, 5303, 5061, 5340, 5375,  501, 4176, 5472, 5514,
        4334, 5548, 3281, 5304, 5588, 5601, 5621, 5642, 4660, 5682, 3109, 5662,
        5781, 5825, 5861, 4987, 5967, 5982, 1209, 6023, 6034, 6038, 5685,  216,
        6200, 6266, 1353, 6149, 6180, 63

In [76]:
y = torch.zeros(wiki.num_nodes, dtype=wiki.y.dtype)
y[wiki.src[wiki.y == 1]] = 1
y

tensor([0, 0, 0,  ..., 0, 0, 0])

In [78]:
(y==1).sum()

tensor(217)

In [79]:
y.shape

torch.Size([9227])